# ENVIRONMENT

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, metrics

In [2]:
sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.3f}'.format

---
# 1. LOAD DATA

In [3]:
path = "data/"

In [4]:
%%time

item_cats = pd.read_csv(path+"item_categories.csv")
items = pd.read_csv(path+"items.csv")
sample_submission = pd.read_csv(path+"sample_submission.csv")
shops = pd.read_csv(path+"shops.csv")
test = pd.read_csv(path+"test.csv")
train = pd.read_csv(path+"sales_train.csv")

print('train: ', train.shape, 'test:',test.shape)
print('item:',items.shape, 'item_cats:', item_cats.shape, 'shops:', shops.shape)

train:  (2935849, 6) test: (214200, 3)
item: (22170, 3) item_cats: (84, 2) shops: (60, 2)
CPU times: total: 1.09 s
Wall time: 3.04 s


In [5]:
sales_train

NameError: name 'sales_train' is not defined

---
# 2. Data analysis

In [8]:
train['date_block_num'].max()

33

In [9]:
train['item_cnt_day'].describe()

count   1,224,439.000
mean            1.321
std             3.313
min           -16.000
25%             1.000
50%             1.000
75%             1.000
max         2,169.000
Name: item_cnt_day, dtype: float64

In [10]:
train['item_cnt_day'].nlargest(25).values

array([2169., 1000.,  624.,  539.,  501.,  500.,  500.,  480.,  405.,
        343.,  299.,  288.,  264.,  255.,  251.,  242.,  230.,  222.,
        217.,  207.,  205.,  205.,  200.,  194.,  187.])

---
# 3. Preprocessing

### 1) 중복값 제거

In [11]:
# 일반 중복값 제거
subset = ['date','date_block_num','shop_id','item_id','item_cnt_day'] #중복검사 해야하는 열 지정
print(train.duplicated(subset=subset).value_counts()) #.duplicated 갯수 세기
train.drop_duplicates(subset=subset, inplace=True) #drop_duplicates

False    1224429
True          10
dtype: int64


#### .duplicated() 사용법
**1. 기본 사용법: dataframe.duplicated()**
선택한 열의 값이 이전 행과 **완전히 일치하는 경우** 해당 행은 중복으로 간주됩니다.

**2. 특정 열 지정: dataframe.duplicated(subset=['column1', 'column2', ...])**
subset 인자를 사용하여 중복을 확인할 특정 열을 지정할 수 있습니다. 지정된 열에서만 값이 이전 행과 일치할 경우 해당 행이 중복으로 간주됩니다.

**3. 중복 처리 방식 변경: dataframe.duplicated(keep='first'/'last'/False)**
keep 인자를 통해 중복된 행 중 어떤 것을 유지할지 결정할 수 있습니다.
keep='first' (기본값): 첫 번째 발생을 제외한 나머지 중복 행을 표시합니다.
keep='last': 마지막 발생을 제외한 나머지 중복 행을 표시합니다.
keep=False: 중복된 모든 행을 표시합니다.

In [12]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [13]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.000,1.000
10,03.01.2013,0,25,2574,399.000,2.000
11,05.01.2013,0,25,2574,399.000,1.000
12,07.01.2013,0,25,2574,399.000,1.000
13,08.01.2013,0,25,2574,399.000,2.000
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.000,1.000
2935845,09.10.2015,33,25,7460,299.000,1.000
2935846,14.10.2015,33,25,7459,349.000,1.000
2935847,22.10.2015,33,25,7440,299.000,1.000


In [14]:
# test 데이터에 없는 데이터 없에기
test_shops = test.shop_id.unique() #unique값 뽑아내기
test_items = test.item_id.unique()
print(test_shops, test_items)

[ 5  4  6  3  2  7 10 12 28 31 26 25 22 24 21 15 16 18 14 19 42 50 49 53
 52 47 48 57 58 59 55 56 36 37 35 38 34 46 41 44 39 45] [ 5037  5320  5233 ... 15757 19648   969]


In [15]:
train = train[train.shop_id.isin(test_shops)]
train = train[train.item_id.isin(test_items)]
print('train:',train.shape)

train: (1224429, 6)


**test에 없는 번호는 제거해주는 이유**
1. 없는거 훈련시켜서 정확도가 떨어질 수도 있기 때문
2. 리소스 효율성 향상

### 2) 시계열 형태로 맞춰주기
train데이터를 살펴보면 모든 shop_id와 item_id에 대해서 정보를 가지고 있지 않다. 데이터가 0이라도 넣어져있어야 하기 때문에 조합으로 만드는 중

In [17]:
from itertools import product

In [19]:
block_shop_combi = pd.DataFrame(list(product(np.arange(34),test_shops)), columns=['date_block_num','shop_id'])
block_shop_combi

,date_block_num,shop_id
0,0,5
1,0,4
2,0,6
3,0,3
4,0,2
...,...,...
1423,33,46
1424,33,41
1425,33,44
1426,33,39


In [28]:
from itertools import product

#모든 조합만들기
block_shop_combi = pd.DataFrame(list(product(np.arange(34),test_shops)), columns=['date_block_num','shop_id'])
shop_item_combi = pd.DataFrame(list(product(test_shops, test_items)), columns = ['shop_id','item_id'])
all_combi = pd.merge(block_shop_combi, shop_item_combi, on=['shop_id'], how = 'inner')
print(len(all_combi), 34 * len(test_shops) * len(test_items))

7282800 7282800


### Sales_train 에서 날짜 맞춰주기
sales_train 에서 [date]
를 년도와 달, 일 포멧으로 맞춰주려고 한다.

In [5]:
sales_train["date"] = pd.to_datetime(sales_train["date"], format = "%d.%m.%Y")

In [6]:
sales_train.iloc[0]

date              2013-01-02 00:00:00
date_block_num                      0
shop_id                            59
item_id                         22154
item_price                      999.0
item_cnt_day                      1.0
Name: 0, dtype: object

### shops 전처리

In [7]:
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


shop을 자세히 보면 똑같은 상점이름이 몇개 있음.
몇개 없으니까 노가다해주자

---

0 : !Якутск Орджоникидзе, 56 фран

57 : Якутск Орджоникидзе, 56

---

1 : !Якутск ТЦ "Центральный" фран

58 : Якутск ТЦ "Центральный"

---

10 : Жуковский ул. Чкалова 39м?	

11 : Жуковский ул. Чкалова 39м²

---

39 : РостовНаДону ТРК "Мегацентр Горизонт"	

40 : РостовНаДону ТРК "Мегацентр Горизонт" Островной	

In [8]:
shops.drop(index=[57, 58, 10, 39], inplace=True)

In [9]:
shops.reset_index(drop=True, inplace=True)
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


##### shop_name에서 city_name 추출
shop_name 중에 하나인 Балашиха ТРК "Октябрь-Киномир"를 뜯어보자.
영어로 번역하면 Balashikha TRK 'Oktyabr-Kinomir 인데

친절한 Chat GPT는 

"Балашиха" (Balashikha): 이는 러시아의 도시 이름입니다. Balashikha는 모스크바 근처에 위치한 도시로 알려져 있습니다.

"ТРК" (TRK): 이는 "Торгово-развлекательный комплекс" (Torgovo-Razvlekatelny Kompleks)의 약자로, 영어로는 "Shopping and Entertainment Complex" 즉, '쇼핑 및 엔터테인먼트 복합 시설'을 의미합니다.

"Октябрь-Киномир" (Oktyabr-Kinomir): 이는 해당 쇼핑 및 엔터테인먼트 복합 시설의 이름입니다. 직역하면 "October-Cinemaworld" 정도가 될 것입니다.

라고 설명해줬다. 즉 도시이름과 상점이름이 합쳐져 있는 형태라는것.
어느 도시에 분포되어있는지도 알려면 떨어뜨려주어야 할 것 같다.

In [10]:
def get_city_name(name :str, prog: re.Pattern = re.compile("[^а-яА-Я.]")) -> str : #-> 데이터타입을 지정해줌
    name = name.lower() #소문자
    name = name.split()[0] # 공백으로 나눠준다음 첫번째 꺼만 가져옴["뉴욕맛나상점"] -> ["뉴욕"]
    name = prog.sub("",name) #아까 prog가 문자만 남기는 거였음. 
    return name